# Compare our values to Nirody et al. 2021
<br>
This notebook = inter-leg coordination <br>
  ... in Nirody this is:<br>

### Figure 4
### Figure S6

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import glob
import gaitFunctions
import plotClip
from scipy import stats
import seaborn as sns

In [ ]:
# navigate to directory that contains the data
os.chdir('/Users/iwoods/OneDrive - Ithaca College/movement_disorder_paper_2023/videos_exemplaris/analyses_summaries/')
for f in sorted(glob.glob('*.xlsx')):
    print(f)

In [ ]:
# specify the data file
data_file = 'iw_9Nov23_exemplaris_inverted_combined.xlsx'